# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
res=requests.get(url)

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup as bs

In [4]:
data=res.text
soup=bs(data, 'html.parser')
links=[]
links2=[]
for link in soup.findAll("a"):
    if 'href' in link.attrs:
        links.append(link.attrs['href'])
print("links totales:",len(links))

for e in links:
    if e.startswith('https:'):
        links2.append(e)
print("unique links:",len(set(links2)))

links totales: 457
unique links: 82


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org'

In [6]:
res=requests.get(domain)
data=res.text
soup=bs(data, 'html.parser')
tot_links=[]
abs_links=[]
rel_links=[]
for link in soup.findAll("a"):
    if 'href' in link.attrs:
        tot_links.append(link.attrs['href'])
print("links totales:",len(tot_links))

abs_links=[e for e in tot_links if e.startswith('http')]
rel_links=['https:'+e for e in tot_links if e.startswith('/') & e.count('%')==0]
print(len(abs_links))
print(len(rel_links))

links totales: 323
2
321


In [7]:
#Lista combinada
tot_links=set(abs_links+rel_links)
len(tot_links)

313

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [8]:
import os

In [9]:
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/sylviac/datamex0320/module-1/lab-parallelization/your-code


In [11]:
#os.mkdir('wikipedia')


In [12]:
os.chdir('wikipedia')

In [13]:
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/sylviac/datamex0320/module-1/lab-parallelization/your-code/wikipedia


## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [14]:
from slugify import slugify

In [2]:
#conda install -c conda-forge python-slugify

In [15]:
import re
link='https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications'
archivo=slugify(link)
nombre=archivo+'.html'
data=requests.get(link).text
soup=bs(data, 'html.parser')
txt=soup.find_all('p')
archivo

'https-en-wikipedia-org-wiki-list-of-wikipedia-mobile-applications'

In [16]:
def index_page(links):
    try:
        data=requests.get(links).text
        soup=bs(data, 'html.parser')
        archivo=slugify(links)
        nombre=archivo+'.html'
        with open(nombre,'w+') as f:
            s=f.write(str(soup))
        return s
    except: pass
        

In [34]:
index_page('https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications')

104084

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [17]:
#conda install -c conda-forge tqdm

In [18]:
import time
from tqdm import tqdm
from tqdm import trange

In [26]:
%%time
total=list(tot_links)
for i in trange(10):
    index_page(total[i])

100%|██████████| 10/10 [00:06<00:00,  1.57it/s]

CPU times: user 1.27 s, sys: 69.4 ms, total: 1.34 s
Wall time: 6.38 s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [27]:
import multiprocessing as mp

In [29]:
mp.cpu_count()

4

In [30]:
%%time
pool = mp.Pool(3)
pool.map(index_page,total[:10])
pool.close()

CPU times: user 10.2 ms, sys: 13.5 ms, total: 23.7 ms
Wall time: 1.82 s


In [31]:
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/sylviac/datamex0320/module-1/lab-parallelization/your-code/wikipedia


In [ ]:
os.chdir('cd')